In [8]:
# modules we'll use
import pandas as pd
import numpy as np

import chardet

# helpful modules
import fuzzywuzzy # if you get error in fuzzu wuzzy not found, pease do pip install fuzzywuzzy and then rerun this step
from fuzzywuzzy import process


# set seed for reproducibility
np.random.seed(0)

In [9]:
with open("./PakistanSuicideAttacks Ver 11 (30-November-2017).csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))

# check what the character encoding might be
print(result) # looks like windows-1252

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [10]:
sucide_attacks = pd.read_csv("./PakistanSuicideAttacks Ver 11 (30-November-2017).csv", encoding = "windows-1252")

In [11]:
sucide_attacks.sample(10)

,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,...,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
90,91,Tuesday-February 5-2008,27 MuHarram 1429 A.H,Working Day,NaN,NA,Quetta,30.20950,67.0182,Baluchistan,...,None,NaN,1.0,NaN,1,1.0,NaN,NaN,-2.370,27.734
254,255,Monday-April 5-2010,20 Raby` al-THaany 1431 A.H.,Working Day,NaN,NaN,Peshawar,34.00430,71.5448,KPK,...,Christian,6.0,8.0,NaN,0,1.0,NaN,NaN,23.135,73.643
283,284,Tuesday-November 30-2010,24 Thw al-Hijjah 1431 A.H,Working Day,NaN,8:45:00 AM,Bannu,32.93500,70.6688,KPK,...,None,2.0,6.0,6.0,22,1.0,12 kg,NaN,9.975,49.955
443,444,Tuesday-May 26-2015,NaN,Working Day,NaN,NaN,Karachi,24.99180,66.9911,Sindh,...,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,33.050,91.490
336,337,Tuesday-January 3-2012,NaN,Working Day,NaN,1:00 PM,Gujrat,32.57370,74.0786,Punjab,...,None,NaN,2.0,NaN,NaN,1.0,NaN,NaN,10.935,51.683
15,16,Thursday-April 28-2005,19 Raby` al-awal 1426 A.H,Working Day,NaN,NaN,Swat,35.22270,72.4258,KPK,...,None,NaN,2.0,NaN,NaN,1.0,NaN,NaN,19.935,67.883
316,317,Sunday-June 5-2011,NaN,Holiday,Weekend,8:45 PM,Nowshehra,34.03220,73.0944,KPK,...,None,18.0,19.0,28.0,45,1.0,8Kg,Lady reading hospital,32.480,90.464
485,486,Friday-June-23-2017,27 RamaDHaan 1438 A.H,Working Day,NaN,1:30 PM,Quetta,33.88423,70.10878,Balochistan,...,Shiite,30.0,85.0,100.0,100+,3.0,75KG,NaN,36.000,97.000
159,160,Thursday-March 26-2009,29 Raby` al-awal 1430 A.H,Working Day,NaN,8:30:00 AM,Tank,32.22600,70.3761,KPK,...,Sunni,7.0,15.0,NaN,25,1.0,NaN,Civil Hospital Tank,14.305,57.749
153,154,Saturday-February 21-2009,26 safar 1430 A.H,Working Day,NaN,NaN,Bannu,32.98610,70.6042,KPK,...,None,NaN,2.0,NaN,NaN,NaN,NaN,NaN,14.420,57.956


In [17]:
cities = sucide_attacks["City"].unique()


cities.sort()
print(cities)  # looking at data you can see incosistencies like  D.I Khan is 2 times, one of the entries has extra space.. this is where fuzzy wuzzy shines.

['ATTOCK' 'Attock ' 'Bajaur Agency' 'Bannu' 'Bhakkar ' 'Buner' 'Chakwal '
 'Chaman' 'Charsadda' 'Charsadda ' 'D. I Khan' 'D.G Khan' 'D.G Khan '
 'D.I Khan' 'D.I Khan ' 'Dara Adam Khel' 'Dara Adam khel' 'Fateh Jang'
 'Ghallanai, Mohmand Agency ' 'Gujrat' 'Hangu' 'Haripur' 'Hayatabad'
 'Islamabad' 'Islamabad ' 'Jacobabad' 'KURRAM AGENCY' 'Karachi' 'Karachi '
 'Karak' 'Khanewal' 'Khuzdar' 'Khyber Agency' 'Khyber Agency ' 'Kohat'
 'Kohat ' 'Kuram Agency ' 'Lahore' 'Lahore ' 'Lakki Marwat' 'Lakki marwat'
 'Lasbela' 'Lower Dir' 'MULTAN' 'Malakand ' 'Mansehra' 'Mardan'
 'Mohmand Agency' 'Mohmand Agency ' 'Mohmand agency'
 'Mosal Kor, Mohmand Agency' 'Multan' 'Muzaffarabad' 'North Waziristan'
 'North waziristan' 'Nowshehra' 'Orakzai Agency' 'Peshawar' 'Peshawar '
 'Pishin' 'Poonch' 'Quetta' 'Quetta ' 'Rawalpindi' 'Sargodha'
 'Sehwan town' 'Shabqadar-Charsadda' 'Shangla ' 'Shikarpur' 'Sialkot'
 'South Waziristan' 'South waziristan' 'Sudhanoti' 'Sukkur' 'Swabi '
 'Swat' 'Swat ' 'Taftan' 'Tangi, 

In [20]:
# removes spaces and trim entries

sucide_attacks['City'] = sucide_attacks['City'].str.lower()
sucide_attacks['City'] = sucide_attacks['City'].str.strip()

In [21]:
cities = sucide_attacks["City"].unique()


cities.sort()
print(cities)

['attock' 'bajaur agency' 'bannu' 'bhakkar' 'buner' 'chakwal' 'chaman'
 'charsadda' 'd. i khan' 'd.g khan' 'd.i khan' 'dara adam khel'
 'fateh jang' 'ghallanai, mohmand agency' 'gujrat' 'hangu' 'haripur'
 'hayatabad' 'islamabad' 'jacobabad' 'karachi' 'karak' 'khanewal'
 'khuzdar' 'khyber agency' 'kohat' 'kuram agency' 'kurram agency' 'lahore'
 'lakki marwat' 'lasbela' 'lower dir' 'malakand' 'mansehra' 'mardan'
 'mohmand agency' 'mosal kor, mohmand agency' 'multan' 'muzaffarabad'
 'north waziristan' 'nowshehra' 'orakzai agency' 'peshawar' 'pishin'
 'poonch' 'quetta' 'rawalpindi' 'sargodha' 'sehwan town'
 'shabqadar-charsadda' 'shangla' 'shikarpur' 'sialkot' 'south waziristan'
 'sudhanoti' 'sukkur' 'swabi' 'swat' 'taftan' 'tangi, charsadda district'
 'tank' 'taunsa' 'tirah valley' 'totalai' 'upper dir' 'wagah' 'zhob']


In [22]:
# Exercise - do the same for Province

sucide_attacks['Province'] = sucide_attacks['Province'].str.lower()
sucide_attacks['Province'] = sucide_attacks['Province'].str.strip()

provinces = sucide_attacks['Province'].unique()

provinces.sort()

provinces

array(['ajk', 'balochistan', 'baluchistan', 'capital', 'fata', 'kpk',
       'punjab', 'sindh'], dtype=object)

In [26]:
# get the top 10 closest matches to "d.i khan"
matches = fuzzywuzzy.process.extract("d.i khan", cities, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# take a look at them
print(matches)

matches = fuzzywuzzy.process.extract("d.i khan", cities, limit=10, scorer=fuzzywuzzy.fuzz.ratio)
print(matches)

[('d. i khan', 100), ('d.i khan', 100), ('d.g khan', 88), ('khanewal', 50), ('sudhanoti', 47), ('hangu', 46), ('kohat', 46), ('dara adam khel', 45), ('chaman', 43), ('mardan', 43)]
[('d.i khan', 100), ('d. i khan', 94), ('d.g khan', 88), ('khanewal', 50), ('sudhanoti', 47), ('hangu', 46), ('kohat', 46), ('dara adam khel', 45), ('chaman', 43), ('mardan', 43)]


In [29]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    
    

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)
    


    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [28]:
replace_matches_in_column(df=sucide_attacks, column='City', string_to_match="d.i khan")

['d.i khan', 'd. i khan']
0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
466    False
467    False
468    False
469    False
470    False
471    False
472    False
473    False
474    False
475    False
476    False
477    False
478    False
479    False
480    False
481    False
482    False
483    False
484    False
485    False
486    False
487    False
488    False
489    False
490    False
491    False
492    False
493    False
494    False
495    False
Name: City, Length: 496, dtype: bool
All done!


In [31]:
# get all the unique values in the 'City' column
cities = sucide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd.g khan', 'd.i khan', 'dara adam khel',
       'fateh jang', 'ghallanai, mohmand agency', 'gujrat', 'hangu',
       'haripur', 'hayatabad', 'islamabad', 'jacobabad', 'karachi',
       'karak', 'khanewal', 'khuzdar', 'khyber agency', 'kohat',
       'kuram agency', 'kurram agency', 'lahore', 'lakki marwat',
       'lasbela', 'lower dir', 'malakand', 'mansehra', 'mardan',
       'mohmand agency', 'mosal kor, mohmand agency', 'multan',
       'muzaffarabad', 'north waziristan', 'nowshehra', 'orakzai agency',
       'peshawar', 'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'],
      dtype=object)

In [32]:
replace_matches_in_column(df=sucide_attacks, column='City', string_to_match="kuram agency")

All done!


In [33]:
cities = sucide_attacks['City'].unique()

# sort them alphabetically and then take a closer look
cities.sort()
cities

array(['attock', 'bajaur agency', 'bannu', 'bhakkar', 'buner', 'chakwal',
       'chaman', 'charsadda', 'd.g khan', 'd.i khan', 'dara adam khel',
       'fateh jang', 'ghallanai, mohmand agency', 'gujrat', 'hangu',
       'haripur', 'hayatabad', 'islamabad', 'jacobabad', 'karachi',
       'karak', 'khanewal', 'khuzdar', 'khyber agency', 'kohat',
       'kuram agency', 'lahore', 'lakki marwat', 'lasbela', 'lower dir',
       'malakand', 'mansehra', 'mardan', 'mohmand agency',
       'mosal kor, mohmand agency', 'multan', 'muzaffarabad',
       'north waziristan', 'nowshehra', 'orakzai agency', 'peshawar',
       'pishin', 'poonch', 'quetta', 'rawalpindi', 'sargodha',
       'sehwan town', 'shabqadar-charsadda', 'shangla', 'shikarpur',
       'sialkot', 'south waziristan', 'sudhanoti', 'sukkur', 'swabi',
       'swat', 'taftan', 'tangi, charsadda district', 'tank', 'taunsa',
       'tirah valley', 'totalai', 'upper dir', 'wagah', 'zhob'],
      dtype=object)